In [8]:
#import tensorflow as tf
import numpy as np
import os
import random
import pickle
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.applications import MobileNet
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from keras.layers import LeakyReLU, ReLU
import json
import cv2
import pandas as pd

dirname = "imgs/test/"

In [9]:
with open('best_params_mobilenet.json') as json_file:
    best_params = json.load(json_file)

def preprocessing(X, img_rows, img_cols, color_type=3):
    
    X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 3)
    X = X.astype('float32')
    X /= 255
    
    return X


def read_image(path, img_rows, img_cols, color_type=3):
  
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  
  resized_image = cv2.resize(image, (img_cols, img_rows))
  
  return resized_image

def load_images(img_rows, img_cols, color_type):
    
    images = []
    path = dirname
    img_names = []

    for img in os.listdir(path):

        try:
            image = read_image(os.path.join(path, img), img_rows, img_cols, color_type)     
            images.append(image)
            img_names.append(img)

        except Exception as e:
            pass

    return images, img_names

def best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet_base):

    model = Sequential()
    model.add(mobilenet_base)
    model.add(Flatten())
    model.add(Dense(best_params["n_units_1"]))
    model.add(ReLU())
    model.add(Dropout(best_params["dropout_rate_1"]))
    model.add(Dense(best_params["n_units_2"]))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(Dropout(best_params["dropout_rate_2"]))
    model.add(Dense(10, activation = 'softmax'))

    model.summary()

    model.compile(optimizer = SGD(lr=best_params["lr"], momentum=best_params["momentum"]), loss='categorical_crossentropy', metrics = ["accuracy"])

    return model
    

In [10]:
img_cols = img_rows = 128
color_type = 3

mobilenet_base = MobileNet(weights=None, include_top=False, input_shape=(img_rows, img_cols, color_type))
model = best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet_base)

X_test, img_names = load_images(128,128,3)
X_test = preprocessing(X_test,128,128,3)

predicitions_test_list = []

for k in range(1,6):
    print(str(k))
    weights_path = os.path.join('best_config_mobilenet','weights_kfold_' + str(k) + '.h5')

    model.load_weights(weights_path)

    predictions_test = model.predict_proba(X_test, verbose=1)
    predicitions_test_list.append(predictions_test)

predicitions_test_list = np.array(predicitions_test_list)
preditions_mean = np.mean(predicitions_test_list, axis = 0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Model)   (None, 4, 4, 1024)        3228864   
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 387)               6340995   
_________________________________________________________________
re_lu_1 (ReLU)               (None, 387)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 387)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 107)               41516     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 107)               0         
__________

In [11]:
preditions_mean.shape

(79726, 10)

In [12]:
len(sorted(os.listdir(dirname))[1:])

79726

In [13]:
df = pd.DataFrame(columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])

df['img'] = img_names

CATEGORIES = ["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
i = 0
for category in CATEGORIES:
    df[category] = preditions_mean[:,i]
    i += 1

In [14]:
df.head(10)

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_60161.jpg,0.000187,0.000003,0.000006,3.135377e-06,0.000030,0.996354,0.000161,0.000201,0.000511,0.002543
1,img_94786.jpg,0.000010,0.000027,0.000342,3.277089e-05,0.000066,0.000416,0.994662,0.000029,0.004302,0.000113
2,img_85853.jpg,0.005850,0.943174,0.000081,4.143559e-03,0.000223,0.000384,0.006745,0.002769,0.031383,0.005246
3,img_36327.jpg,0.000423,0.877156,0.000107,7.674249e-04,0.000058,0.000043,0.009092,0.003534,0.106686,0.002132
4,img_39014.jpg,0.521233,0.002783,0.001067,1.214497e-02,0.002350,0.446178,0.000395,0.000162,0.002096,0.011591
5,img_84595.jpg,0.015025,0.000013,0.000614,4.558388e-05,0.000409,0.000961,0.000003,0.000303,0.002286,0.980340
6,img_57915.jpg,0.002323,0.000442,0.000274,9.374436e-01,0.001145,0.057816,0.000100,0.000131,0.000211,0.000115
7,img_37039.jpg,0.000637,0.000007,0.000031,6.566190e-04,0.000058,0.998316,0.000017,0.000016,0.000144,0.000118
8,img_11940.jpg,0.000172,0.002948,0.732462,1.132443e-04,0.000469,0.007314,0.190202,0.044791,0.017689,0.003839
9,img_20545.jpg,0.000003,0.000007,0.999864,7.683667e-07,0.000004,0.000006,0.000047,0.000015,0.000047,0.000006


In [15]:
df.to_csv("submission_mobilenet.csv",index=False)